<a href="https://colab.research.google.com/github/PhilippMatthes/diplom/blob/master/src/shl-deep-learning-timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a deep CNN to directly classify SHL timeseries data

In [1]:
# Get needed auxiliary files for colab
!git clone https://github.com/philippmatthes/diplom
%cd /content/diplom/src
!mkdir shl-dataset
!wget -nc -O shl-dataset/challenge-2019-train_torso.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
!wget -nc -O shl-dataset/challenge-2019-train_bag.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
!wget -nc -O shl-dataset/challenge-2019-train_hips.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_hips.zip
!wget -nc -O shl-dataset/challenge-2020-train_hand.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2020/challenge-2020-train_hand.zip

Cloning into 'diplom'...
remote: Enumerating objects: 1362, done.
remote: Counting objects: 100% (699/699), done.
remote: Compressing objects: 100% (434/434), done.
remote: Total 1362 (delta 334), reused 578 (delta 227), pack-reused 663
Receiving objects: 100% (1362/1362), 27.91 MiB | 22.66 MiB/s, done.
Resolving deltas: 100% (710/710), done.
/content/diplom/src
--2021-08-06 06:39:38--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl-dataset.org)|37.187.125.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852446972 (5.5G) [application/zip]
Saving to: ‘shl-dataset/challenge-2019-train_torso.zip’

shl-dataset/challen 100%[===================>]   5.45G  8.99MB/s    in 10m 37s 

2021-08-06 06:50:16 (8.76 MB/s) - ‘shl-dataset/challenge-2019-train_torso.zip’ saved [5852446972/5852446972]

--2021-08-06 06:50:1

In [1]:
# Switch to src dir and select tensorflow
%cd /content/diplom/src
%tensorflow_version 1.x

/content/diplom/src
TensorFlow 1.x selected.


In [2]:
from pathlib import Path

MODEL_DIR = Path('models/shl-model-sparse-cat-crossentropy-resnet.h5')

In [3]:
# Define all datasets to train our model on

DATASET_DIRS = [
    Path('shl-dataset/challenge-2019-train_torso.zip'),
    Path('shl-dataset/challenge-2019-train_bag.zip'),
    Path('shl-dataset/challenge-2019-train_hips.zip'),
    Path('shl-dataset/challenge-2020-train_hand.zip'),
]

In [4]:
# Create scalers that we will fit on our data

from sklearn.preprocessing import PowerTransformer

acc_scaler = PowerTransformer()
mag_scaler = PowerTransformer()
gyr_scaler = PowerTransformer()

In [5]:
from tensorflow import keras

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 698827794333744589, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 7408764812135247460
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13164128917316713892
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15964005991
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1661979432569129427
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [ ]:
import numpy as np

from tools.dataset import load_zipped_shl_dataset
from tools.export import export_power_transformer

from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight

# Join all datasets
acc_mag_conc = None
mag_mag_conc = None
gyr_mag_conc = None
y_conc = None

for dataset_dir in DATASET_DIRS:
    # Load dataset from zip file into temporary directory
    dataset = load_zipped_shl_dataset(dataset_dir, tqdm=tqdm)
    if acc_mag_conc is None:
        acc_mag_conc = dataset.acc_mag
    else:
        acc_mag_conc = np.concatenate((acc_mag_conc, dataset.acc_mag), axis=0)
    if mag_mag_conc is None:
        mag_mag_conc = dataset.mag_mag
    else:
        mag_mag_conc = np.concatenate((mag_mag_conc, dataset.mag_mag), axis=0)
    if gyr_mag_conc is None:
        gyr_mag_conc = dataset.gyr_mag
    else:    
        gyr_mag_conc = np.concatenate((gyr_mag_conc, dataset.gyr_mag), axis=0)
    if y_conc is None:
        y_conc = dataset.labels
    else:
        y_conc = np.concatenate((y_conc, dataset.labels), axis=0)
    del dataset
    

In [7]:

# Sanity checks
assert not np.isnan(acc_mag_conc).any()
assert not np.isnan(mag_mag_conc).any()
assert not np.isnan(gyr_mag_conc).any()

# Fit and export scalers
acc_mag_scaled = acc_scaler.fit_transform(acc_mag_conc)
del acc_mag_conc
mag_mag_scaled = mag_scaler.fit_transform(mag_mag_conc)
del mag_mag_conc
gyr_mag_scaled = gyr_scaler.fit_transform(gyr_mag_conc)
del gyr_mag_conc

export_power_transformer(acc_scaler, 'models/acc-transformer.json')
export_power_transformer(mag_scaler, 'models/mag-transformer.json')
export_power_transformer(gyr_scaler, 'models/gyr-transformer.json')

# Prepare training data
X = np.stack([
    acc_mag_scaled,
    mag_mag_scaled, 
    gyr_mag_scaled,
], axis=2)
del acc_mag_scaled
del mag_mag_scaled
del gyr_mag_scaled

# Compute class weights for unbiased training
class_weights = compute_class_weight('balanced', classes=np.unique(y_conc), y=y_conc)
class_weights = dict(zip(np.unique(y_conc), class_weights)) # Keras adaption

In [18]:
# Create our model

from tools.architectures import make_resnet
from tools.dataset import shl_dataset_label_order

OVERRIDE_MODEL = True

if MODEL_DIR.is_file() and not OVERRIDE_MODEL:
    # Load model for transfer learning
    model = keras.models.load_model(MODEL_DIR)
else:
    # Create model
    model = make_resnet(
        input_shape=[500, 3], # Magnitudes of ACC, MAG and Gyr, 500 data points each 
        output_classes=len(shl_dataset_label_order),
        n_feature_maps=256
    )
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['acc']
    )

In [19]:
# Train model
callbacks = [
    keras.callbacks.CSVLogger(f'{MODEL_DIR}.log', append=MODEL_DIR.is_file()),
    keras.callbacks.ModelCheckpoint(
        str(MODEL_DIR), save_best_only=True, monitor='val_loss', verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001, verbose=1
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1),
]
model.fit(
    X, 
    y_conc,
    epochs=100, 
    batch_size=32,
    class_weight=class_weights, 
    callbacks=callbacks,
    validation_split=0.1,
    verbose=1,
    shuffle=True
)

Train on 705859 samples, validate on 78429 samples
Epoch 1/100
705856/705859 [============================>.] - ETA: 0s - loss: 0.6539 - acc: 0.7155
Epoch 00001: val_loss improved from inf to 0.70941, saving model to models/shl-model-sparse-cat-crossentropy-resnet.h5
705859/705859 [==============================] - 3226s 5ms/sample - loss: 0.6539 - acc: 0.7155 - val_loss: 0.7094 - val_acc: 0.7010
Epoch 2/100
705856/705859 [============================>.] - ETA: 0s - loss: 0.4630 - acc: 0.8057
Epoch 00002: val_loss improved from 0.70941 to 0.70846, saving model to models/shl-model-sparse-cat-crossentropy-resnet.h5
705859/705859 [==============================] - 3208s 5ms/sample - loss: 0.4630 - acc: 0.8057 - val_loss: 0.7085 - val_acc: 0.7090
Epoch 3/100
705856/705859 [============================>.] - ETA: 0s - loss: 0.4115 - acc: 0.8295
Epoch 00003: val_loss improved from 0.70846 to 0.65725, saving model to models/shl-model-sparse-cat-crossentropy-resnet.h5
705859/705859 [===========

KeyboardInterrupt: ignored